<a href="https://colab.research.google.com/github/bagustris/ravdess_song/blob/main/ravdess_song_sd_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition with LSTM 
Acoustic features are loaded directly from data directory

In [3]:
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [4]:
#!git clone https://github.com/bagustris/ravdess_song.git

In [5]:
#cd ravdess_song/

In [6]:
# assign hsf to X, lab to y; reshape X for LSTM
X = np.load('data/x.npy', allow_pickle=True)
y = np.load('data/y.npy', allow_pickle=True)

In [7]:
# reshape x untuk lstm
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [8]:
# split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [9]:
# functio to build LSTM model
def model_lstm():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [11]:
# create the model
model = model_lstm()
print(model.summary())
# hasil print model. shape dense hanya 6 sesuai dengan outputnya yaitu 6 emosi dalam song

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 1, 386)            1544      
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 256)            658432    
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
lstm_5 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                

In [12]:
# train the Dense model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/100
57/57 [==============================] - 54s 119ms/step - loss: 1.4292 - accuracy: 0.4374
Epoch 2/100
57/57 [==============================] - 7s 130ms/step - loss: 0.6479 - accuracy: 0.7571
Epoch 3/100
57/57 [==============================] - 7s 126ms/step - loss: 0.4609 - accuracy: 0.8286
Epoch 4/100
57/57 [==============================] - 8s 134ms/step - loss: 0.3339 - accuracy: 0.8670
Epoch 5/100
57/57 [==============================] - 9s 155ms/step - loss: 0.3223 - accuracy: 0.8846
Epoch 6/100
57/57 [==============================] - 7s 119ms/step - loss: 0.2495 - accuracy: 0.9132
Epoch 7/100
57/57 [==============================] - 8s 142ms/step - loss: 0.2608 - accuracy: 0.9088
Epoch 8/100
57/57 [==============================] - 7s 128ms/step - loss: 0.1781 - accuracy: 0.9374
Epoch 9/100
57/57 [==============================] - 8s 132ms/step - loss: 0.1661 - accuracy: 0.9396
Epoch 10/100
57/57 [==============================] - 7s 129ms/step - loss: 0.1476 - accur

In [13]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

7/7 [==============================] - 7s 25ms/step - loss: 0.5617 - accuracy: 0.8627
Loss:  0.5616524815559387 --> Accuracy:  0.8627451062202454
